# gebrauchte Bücher aus Goodreads to-read liste suchen

In [1]:
import numpy as np
import csv

In [2]:
from matplotlib import pyplot as plt

## goodreads CSV-Datei parsen

In [3]:
import os
csv_path = os.path.normpath(r"C:\Users\simon\Downloads\goodreads_library_export.csv")
import pandas as pd

df = pd.read_csv(csv_path, index_col='Book Id',
dtype={"Title": str, "Author": str, "Author l-f": str, "Additional Authors": str, "ISBN":str,"ISBN-13":str, "Publisher": str, "Binding": str, 'Bookshelves': 'category', 'Exclusive Shelf': 'category', },
parse_dates=["Original Publication Year", 'Date Read', 'Date Added', "Year Published"])
df.head(5)

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,...,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,
42242793,Kurze Antworten auf große Fragen,Stephen Hawking,"Hawking, Stephen",Hainer Kober,"=""""","=""""",0,4.30,Klett-Cotta,Kindle Edition,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
41160292,Exhalation: Stories,Ted Chiang,"Chiang, Ted",NaN,"=""""","=""""",4,4.27,Knopf,Kindle Edition,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
9778945,The Shallows: What the Internet Is Doing to Ou...,Nicholas Carr,"Carr, Nicholas",NaN,"=""0393339750""","=""9780393339758""",0,3.89,W. W. Norton Company,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
23995249,The Invention of Nature: Alexander von Humbold...,Andrea Wulf,"Wulf, Andrea",NaN,"=""038535066X""","=""9780385350662""",0,4.24,Knopf,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
52668915,"Entangled Life: How Fungi Make Our Worlds, Cha...",Merlin Sheldrake,"Sheldrake, Merlin",NaN,"=""0525510311""","=""9780525510314""",0,4.42,Random House,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [4]:
df['ISBN'] = df['ISBN'].str.split('\"').str.get(1)
df['ISBN13'] = df['ISBN13'].str.split('\"').str.get(1)
df.head()

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,...,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,
42242793,Kurze Antworten auf große Fragen,Stephen Hawking,"Hawking, Stephen",Hainer Kober,,,0,4.30,Klett-Cotta,Kindle Edition,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
41160292,Exhalation: Stories,Ted Chiang,"Chiang, Ted",NaN,,,4,4.27,Knopf,Kindle Edition,...,NaN,1,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
9778945,The Shallows: What the Internet Is Doing to Ou...,Nicholas Carr,"Carr, Nicholas",NaN,0393339750,9780393339758,0,3.89,W. W. Norton Company,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
23995249,The Invention of Nature: Alexander von Humbold...,Andrea Wulf,"Wulf, Andrea",NaN,038535066X,9780385350662,0,4.24,Knopf,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
52668915,"Entangled Life: How Fungi Make Our Worlds, Cha...",Merlin Sheldrake,"Sheldrake, Merlin",NaN,0525510311,9780525510314,0,4.42,Random House,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [5]:
to_read = df[df['Exclusive Shelf'] == 'to-read']


In [6]:
import datetime as dt

In [7]:
to_read.sort_values('Date Added', ascending=False, inplace=True)
to_read_2021 = to_read[to_read['Date Added'] > pd.Timestamp('2020-01-01')]

In [8]:
to_read_2021.head()

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,...,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,
9778945,The Shallows: What the Internet Is Doing to Ou...,Nicholas Carr,"Carr, Nicholas",NaN,0393339750,9780393339758,0,3.89,W. W. Norton Company,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
52668915,"Entangled Life: How Fungi Make Our Worlds, Cha...",Merlin Sheldrake,"Sheldrake, Merlin",NaN,0525510311,9780525510314,0,4.42,Random House,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
26150770,"The Big Picture: On the Origins of Life, Meani...",Sean Carroll,"Carroll, Sean",NaN,0525954821,9780525954828,0,4.18,Dutton,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
23995249,The Invention of Nature: Alexander von Humbold...,Andrea Wulf,"Wulf, Andrea",NaN,038535066X,9780385350662,0,4.24,Knopf,Hardcover,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
53687124,"Chasing the Light: Writing, Directing, and Sur...",Oliver Stone,"Stone, Oliver",NaN,,,0,4.07,Houghton Mifflin Harcourt,Kindle Edition,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


## Jetzt Suchmaschine mittels BeatifulSoup scrapen um Bestpreise für Bücher zu finden

In [9]:
from bs4 import BeautifulSoup
import requests

In [10]:
#Testbuch
book = to_read_2021.iloc[0]
book

Title                         The Shallows: What the Internet Is Doing to Ou...
Author                                                            Nicholas Carr
Author l-f                                                       Carr, Nicholas
Additional Authors                                                          NaN
ISBN                                                                 0393339750
ISBN13                                                            9780393339758
My Rating                                                                     0
Average Rating                                                             3.89
Publisher                                                 W. W. Norton  Company
Binding                                                               Paperback
Number of Pages                                                           280.0
Year Published                                              2011-01-01 00:00:00
Original Publication Year               

In [11]:
url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/isbn=0393339750?sortOrder=preis_total'
search_url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/'
base_url = 'https://www.booklooker.de'
query_parameters = {
    'isbn': book['ISBN13'],
    'sortOrder': 'preis_total'
}

In [12]:
page = requests.get(url)
soup = BeautifulSoup(page.content)

table class="resultlist_products"

tbody

tr id="article_*ID*" class="articleRow resultlist_products*product*" ODER

articleRow resultlist_products*premium*

In [13]:
results = soup.find(class_='resultlist_products')
products = results.find_all('tr', class_='articleRow')


In [14]:
entries = []
for idx, product in enumerate(products):
    product_info = product.find("td", class_="resultlist_productsimage")
    price_info = product.find("td", class_="resultlist_productsright")
    title_info =product_info.find('h2', class_='unformated')
    book_title = title_info.find('span', class_='artikeltitel')
    link = base_url + title_info.a.attrs['href']

    author = product_info.find('span', class_='notranslate')
    condition = str(product_info.find(string=lambda text: 'zustand' in text.lower()))
    prefix = 'Zustand: '
    condition = condition[len(prefix):] if condition.startswith(prefix) else condition
    price = price_info.find('span', class_='price')
    row = [book_title.text.strip(), author.text.strip(), price.text.strip()[:-2], condition.strip(), link]
    entries.append(row)
    # print(f"[{idx}]: title: {book_title.text.strip()}, author: {author.text.strip()}, price = {price.text.strip()}, state: {state.strip()}), link: {link}")


In [15]:
web = pd.DataFrame(entries, columns=['Title', 'Author', 'Price', 'Condition', 'URL'])
web.head()

,Title,Author,Price,Condition,URL
0,"Carr, N: Shallows","Carr, N: Shallows","3,21",gebraucht; mittelmäßig,https://www.booklooker.de/B%C3%BCcher/Nicholas...


Build Search URL

In [27]:
from urllib.parse import urlencode, urljoin, quote_plus
def build_booklooker_url(**kwargs):
    isbn13 = kwargs.get('ISBN13', "")
    isbn = kwargs.get('ISBN', "")
    title = kwargs.get('Title', "")
    author = kwargs.get('Author', "")
    
    search_url = 'https://www.booklooker.de/B%C3%BCcher/Angebote/'
    payload = {'sortOrder': "preis_total"}
    if len(isbn13) > 0:
        payload['isbn'] = isbn13
    elif len(isbn) > 0:
        payload['isbn'] = isbn
    else:
        payload['autor'] = author
        payload['titel'] = title
        print(f'ISBN not found, using: {author}: {title}')

    url = build_url(search_url, payload)
    return url

def build_abebooks_url(**kwargs):
    #TODO
    raise NotImplementedError('not yet finished.')
    search_url = 'https://www.abebooks.de/servlet/SearchResults?sts=t&cm_sp=SearchF-_-home-_-Results&an=Author&tn=Title&kn=&isbn='
    isbn13 = kwargs.get('ISBN13', "")
    isbn = kwargs.get('ISBN', "")
    title = kwargs.get('Title', "")
    author = kwargs.get('Author', "")
    
    payload = {'sortOrder': "preis_total"}
    if len(isbn13) > 0:
        payload['isbn'] = isbn13
    elif len(isbn) > 0:
        payload['isbn'] = isbn
    else:
        payload['an'] = author
        payload['tn'] = title
        print(f'ISBN not found, using: {author}: {title}')

    url = build_url(search_url, payload)
    return url

def build_url(base_url, payload):
    result = urlencode(payload, quote_via=quote_plus)
    url = urljoin(base_url, result)
    return url

def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content.decode('utf-8', 'ignore'))
    return soup

In [28]:
def get_booklooker_results(soup, limit=10, as_df=False):
    entries = []
    results = soup.find(class_='resultlist_products')
    if results is None:
        print('no resultlist found! returning.')
        return entries
    products = results.find_all('tr', class_='articleRow')
    for idx, product in enumerate(products):
        if idx > limit:
            break
        product_info = product.find("td", class_="resultlist_productsimage")
        title_info =product_info.find('h2', class_='unformated')
        book_title = title_info.find('span', class_='artikeltitel notranslate')
        link = base_url + title_info.a.attrs['href']

        author_info =product_info.find('h3', class_='unformated')
        author = author_info.find('span', class_='notranslate')

        condition = str(product_info.find(string=lambda text: 'zustand' in text.lower()))
        prefix = 'Zustand: '
        condition = condition[len(prefix):] if condition.startswith(prefix) else condition

        price_info = product.find("div", class_="productPrices")
        prices = price_info.find_all(string=lambda text: '€' in text.lower())
        price_list = [s[:-1].strip() for s in prices]
        price = price_list[0] if len(price_list) > 0 else ""
        vsk = price_list[1] if len(price_list) > 1 else ""
        
        row = [book_title.text.strip(), author.text.strip(), price, vsk, condition.strip(), link]
        entries.append(row)
    if as_df:
        return pd.DataFrame(entries, columns=['Title', 'Author', 'Price', 'VSK', 'Condition', 'URL'])
    else:
        return entries


## Bringing it all together

In [249]:
def get_sales_info_df(df, limit=10, as_df=True):
    url = build_booklooker_url(**df)
    soup = get_soup(url)
    search_results = get_booklooker_results(soup, limit=limit, as_df=as_df)
    if as_df:
        return search_results
    else:
        return np.array(search_results)

In [250]:
def get_sales_info(title, isbn, isbn13, limit=10, as_df=True):
    isbn_ = isbn13 if len(isbn13) > 0 else isbn
    if len(isbn_) <= 0:
        print(f'{title}: no ISBN found!')
        return []
        # raise KeyError('No ISBN Found')
    d = {'ISBN': isbn_}
    url = build_booklooker_url(**d)
    soup = get_soup(url)
    search_results = get_booklooker_results(soup, limit=limit, as_df=as_df)
    return np.array(search_results)

In [251]:
from tqdm import tqdm
# sales_info = []
my_df = to_read_2021.iloc[:20]
from tqdm import tqdm
# Create and register a new `tqdm` instance with `pandas`
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

# Now you can use `progress_apply` instead of `apply`
sales_infos = my_df.progress_apply(get_sales_info_df, axis=1)


 35%|███▌      | 7/20 [00:01<00:03,  3.30it/s]no resultlist found! returning.
ISBN not found, using: Katharina Nocun: Die Daten, die ich rief: Wie wir unsere Freiheit an Großkonzerne verkaufen
100%|██████████| 20/20 [00:08<00:00,  2.40it/s]


In [252]:
sales_infos.head()
# nd = np.array(sales_infos)

Book Id
9778945                    Title         Author Price   VS...
52668915                                                   []
26150770                                                  ...
23995249                                                  ...
53687124                                                   []
dtype: object

In [281]:
sales_infos.iloc[3].iloc[0]

Title        The Invention of Nature - Alexander von Humbol...
Author                                             Andrea Wulf
Price                                                    29,00
VSK                                                       5,50
Condition                                              wie neu
URL          https://www.booklooker.de/B%C3%BCcher/Andrea-W...
Name: 0, dtype: object

Erstmal speichern

In [257]:
sales_infos.to_pickle('sales_infos_2021_20stck.pkl')

In [295]:
sales_arr = sales_infos.to_numpy()
sales_df = pd.DataFrame(sales_arr)

In [298]:
sales_df.iloc[2].iloc[0]

,Title,Author,Price,VSK,Condition,URL
0,The Big Picture,Sean Carroll,"17,99",,None,https://www.booklooker.de/B%C3%BCcher/Sean-Car...
1,The Big Picture,Sean Carroll,"19,95",,None,https://www.booklooker.de/B%C3%BCcher/Sean-Car...
2,"The Big Picture | On the Origins of Life, Mean...","Carroll, Sean B.","22,90",,None,https://www.booklooker.de/B%C3%BCcher/Sean-B-C...
3,The Big Picture,Sean B. Carroll,"23,85",,None,https://www.booklooker.de/B%C3%BCcher/Sean-B-C...
4,"The Big Picture | On the Origins of Life, Mean...","Carroll, Sean B.","24,90",,None,https://www.booklooker.de/B%C3%BCcher/Sean-B-C...
5,The Big Picture,Sean B. Carroll,"25,18",,None,https://www.booklooker.de/B%C3%BCcher/Sean-B-C...
6,The Big Picture,Sean B. Carroll,"25,18",,None,https://www.booklooker.de/B%C3%BCcher/Sean-B-C...
7,The Big Picture,Sean B. Carroll,"26,50",,None,https://www.booklooker.de/B%C3%BCcher/Sean-B-C...
8,The Big Picture,Sean B. Carroll,"26,50",,None,https://www.booklooker.de/B%C3%BCcher/Sean-B-C...
9,The Big Picture,Sean B. Carroll,"26,50",,None,https://www.booklooker.de/B%C3%BCcher/Sean-B-C...


## TODO daraus jetzt ein MultiIndex basteln

In [184]:
# sales_df = sales_infos.to_frame()   # series to dataframe
# sales_arr = sales_df.to_numpy()     # dataframe to numpy-array
# print(f"shape before: {sales_arr.shape}")
# print(f"shape[2] before: {sales_arr[2].shape}")
# print(f"shape[2][0] before: {sales_arr[2][0].shape}")
# sales_arr = np.squeeze(sales_arr)   # remove empty-layer in numpy-array
# print(f"shape after squeeze: {sales_arr.shape}")
# print(f"shape[2] after: {sales_arr[2].shape}")
# print(f"shape[2][0] after: {sales_arr[2][0].shape}")
# #Don't remove the empty ones, so the order stays the same (book 1 = bookid[1]...)
# # sales_arr = sales_arr[np.array([y.size>0 for y in sales_arr])] # remove empty results
# # print(f"removed empty lists: {sales_arr.shape}")

shape before: (20, 1)
shape[2] before: (1,)
shape[2][0] before: (11, 6)
shape after squeeze: (20,)
shape[2] after: (11, 6)
shape[2][0] after: (6,)


jetzt ist alles gecleaned.
Übrig: N Bücher, mit M Suchergebnissen und den jeweils 6 Spalten: (N,M,6) 

In [146]:
test_arr = sales_arr[1]

In [162]:
test_arr.T[5]

array(['https://www.booklooker.de/B%C3%BCcher/Sean-Carroll+The-Big-Picture/id/A02u759P01ZZ1',
       'https://www.booklooker.de/B%C3%BCcher/Sean-Carroll+The-Big-Picture/id/A02u5KTL01ZZw',
       'https://www.booklooker.de/B%C3%BCcher/Sean-B-Carroll+The-Big-Picture-On-the-Origins-of-Life-Meaning-and-the-Universe-Itself-Sean-B-Carroll/id/A02u5SGv01ZZV',
       'https://www.booklooker.de/B%C3%BCcher/Sean-B-Carroll+The-Big-Picture/id/A02u5DLB01ZZz',
       'https://www.booklooker.de/B%C3%BCcher/Sean-B-Carroll+The-Big-Picture-On-the-Origins-of-Life-Meaning-and-the-Universe-Itself-Sean-B-Carroll/id/A02u5R2p01ZZW',
       'https://www.booklooker.de/B%C3%BCcher/Sean-B-Carroll+The-Big-Picture/id/A02u5FeG01ZZn',
       'https://www.booklooker.de/B%C3%BCcher/Sean-B-Carroll+The-Big-Picture/id/A02u5E3101ZZa',
       'https://www.booklooker.de/B%C3%BCcher/Sean-B-Carroll+The-Big-Picture/id/A02u5FXZ01ZZ1',
       'https://www.booklooker.de/B%C3%BCcher/Sean-B-Carroll+The-Big-Picture/id/A02u8qWy01ZZ1',


In [192]:
sales_df.head(1)

,0
Book Id,
9778945,"[[Carr, N: Shallows, Nicholas Carr, 3,21, 2,50..."


In [188]:
my_df.head(1)

,Title,Author,Author l-f,Additional Authors,ISBN,ISBN13,My Rating,Average Rating,Publisher,Binding,...,Private Notes,Read Count,Recommended For,Recommended By,Owned Copies,Original Purchase Date,Original Purchase Location,Condition,Condition Description,BCID
Book Id,,,,,,,,,,,,,,,,,,,,,
9778945,The Shallows: What the Internet Is Doing to Ou...,Nicholas Carr,"Carr, Nicholas",NaN,0393339750,9780393339758,0,3.89,W. W. Norton Company,Paperback,...,NaN,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [164]:
idx = pd.MultiIndex.from_arrays(test_arr.T, names=['Title', 'Author', 'Price', 'VSK', 'Condition', 'URL'])


In [191]:
ids = my_df.index
ids

Int64Index([ 9778945, 52668915, 26150770, 23995249, 53687124, 50403471,
            39778359, 44492286, 29780253,    24113, 30335077,   112537,
               24861, 55361205,   217381, 41036165, 52578297, 37542581,
            53179105, 26153173],
           dtype='int64', name='Book Id')

In [201]:
result_rows = np.array(["result_%d" % i for i in range(10)]) #Patients

cols = ['Title', 'Author', 'Price', 'VSK', 'Condition', 'URL']

In [204]:
# Create the MultiIndex from years, samples and patients.

midx = pd.MultiIndex.from_product([ids, result_rows], names=['Book Id', 'Search Results'])
# Note that Cartesian product order is the same as the 
# C-order used by default in ``reshape``.
midx

MultiIndex([( 9778945, 'result_0'),
            ( 9778945, 'result_1'),
            ( 9778945, 'result_2'),
            ( 9778945, 'result_3'),
            ( 9778945, 'result_4'),
            ( 9778945, 'result_5'),
            ( 9778945, 'result_6'),
            ( 9778945, 'result_7'),
            ( 9778945, 'result_8'),
            ( 9778945, 'result_9'),
            ...
            (26153173, 'result_0'),
            (26153173, 'result_1'),
            (26153173, 'result_2'),
            (26153173, 'result_3'),
            (26153173, 'result_4'),
            (26153173, 'result_5'),
            (26153173, 'result_6'),
            (26153173, 'result_7'),
            (26153173, 'result_8'),
            (26153173, 'result_9')],
           names=['Book Id', 'Search Results'], length=200)

(11, 6)

In [215]:
#A_3D muss shape (20x10)x6 = 200x6 haben
# Create search-result data for all columns, and add the MultiIndex.
web_data = pd.DataFrame(index = midx,columns = cols)


In [221]:
# sales_arr.shape

for book_idx, book in enumerate(sales_arr)
    for result_idx, search_result in enumerate(book):
        #search_result[0] - search_result[5] sind dann die cols
# sales_arr[book_no][result_no]

(20,)

In [231]:
test_df = pd.DataFrame(sales_arr, index=ids, columns=cols)

ValueError: Shape of passed values is (20, 1), indices imply (20, 6)

In [232]:
ids.shape

(20,)

In [169]:
idx[2]

('The Big Picture | On the Origins of Life, Meaning, and the Universe Itself | Sean B. Carroll | Buch | Englisch | 2016',
 'Carroll, Sean B.',
 '22,90',
 '',
 'None',
 'https://www.booklooker.de/B%C3%BCcher/Sean-B-Carroll+The-Big-Picture-On-the-Origins-of-Life-Meaning-and-the-Universe-Itself-Sean-B-Carroll/id/A02u5SGv01ZZV')

In [172]:
s = pd.Series(np.random.randn(11), index=idx)   #Eintrag wäre der Titel bzw. die BookID und index dann die Einträge.. .naja is ja eigentlich falschrum

In [176]:
s.head()

Title                                                                                                                  Author            Price  VSK  Condition  URL                                                                                                                                                        
The Big Picture                                                                                                        Sean Carroll      17,99       None       https://www.booklooker.de/B%C3%BCcher/Sean-Carroll+The-Big-Picture/id/A02u759P01ZZ1                                                                           -1.044089
                                                                                                                                         19,95       None       https://www.booklooker.de/B%C3%BCcher/Sean-Carroll+The-Big-Picture/id/A02u5KTL01ZZw                                                                            1.241112
The Big Picture | On the Ori

ggf die shape umkehren? N Einträge (rows) mit je 6 Spalten und zu jeder Spalte dann ne Variable Liste von Einträgen?

In [177]:
np.random.seed(1618033)

#Set 3 axis labels/dims
years = np.arange(2000,2010) #Years 
samples = np.arange(0,20) #Samples
patients = np.array(["patient_%d" % i for i in range(0,3)]) #Patients

#Create random 3D array to simulate data from dims above
A_3D = np.random.random((years.size, samples.size, len(patients))) #(10, 20, 3)

# Create the MultiIndex from years, samples and patients.
midx = pd.MultiIndex.from_product([years, samples, patients])

# Create sample data for each patient, and add the MultiIndex.
patient_data = pd.DataFrame(np.random.randn(len(midx), 3), index = midx)


years ^= BookID
Sample ^= Suchergebnisse (z.B. max 10)


In [179]:
patient_data.head()

0         1         2
2000 0 patient_0  0.733891 -1.081459  1.344767
       patient_1  0.183899  0.367003 -1.043212
       patient_2 -0.233493  0.424136  0.035358
     1 patient_0  1.764374  3.095941  1.680118
       patient_1 -0.146588  1.167771  0.218991

In [180]:
patient_data.unstack(level=0).head()  # Years.


0                                                    \
                 2000      2001      2002      2003      2004      2005   
0 patient_0  0.733891  0.313232 -0.383357  1.750069 -1.055323  0.038683   
  patient_1  0.183899  0.130183 -2.402050 -0.195703 -1.295543  0.179823   
  patient_2 -0.233493  1.069902 -0.414003 -0.331743 -0.148233 -0.816054   
1 patient_0  1.764374  0.706257 -0.141564  1.134810  1.412329  0.094802   
  patient_1 -0.146588 -0.396285 -0.050046  0.492005  0.078164 -2.426065   

                                                     ...         2            \
                 2006      2007      2008      2009  ...      2000      2001   
0 patient_0 -0.795319  0.866563 -0.072328 -0.761547  ...  1.344767  0.639748   
  patient_1 -0.256294 -0.138591  0.401505  0.941614  ... -1.043212  1.291971   
  patient_2 -1.169933  0.485801  0.506001  0.994103  ...  0.035358  0.711121   
1 patient_0 -0.129987 -0.721118 -1.105769 -0.387461  ...  1.680118 -0.215912   
  patient_1  1.710801  0.130949  1.059029  0.570667  ...  0.218991 -0.352376   

                                                                         \
                 2002      2003      2004      2005      2006      2007   
0 patient_0  0.452344  0.745117 -1.557094 -0.809781 -0.506905 -0.871313   
  patient_1  0.581809 -0.239743 -0.295652  0.743647 -0.777149 -1.168015   
  patient_2  1.532766  0.785486 -0.347170 -1.143815 -1.146629 -0.393508   
1 patient_0 -0.881041  0.003546  0.718917 -0.101255  0.613290  0.870485   
  patient_1 -0.082334  0.033250  0.261488  0.808276  1.909453 -0.560593   

                                 
                 2008      2009  
0 patient_0  1.425371 -0.589768  
  patient_1  0.503152 -0.060225  
  patient_2 -0.648646 -0.614766  
1 patient_0 -0.871202 -0.883699  
  patient_1  0.647894 -0.454246  

[5 rows x 30 columns]

awesome, this part essentially makes it 4D np.random.randn(len(midx), 3)! The unstacking order gets a little confusing on how the index is formed #level=0: index = (sample,patient); col = (data,year) #level=1: index = (year,patient); col = (data,sample) #level=2: index = (year,sample); col = (data, patient) – O.rka Apr 21 '16 at 7:38


### Variante 2

In [181]:
#Set 3 axis labels/dims
years = np.arange(2000,2010) #Years
samples = np.arange(0,20) #Samples
patients = np.array(["patient_%d" % i for i in range(0,3)]) #Patients

#Create random 3D array to simulate data from dims above
A_3D = np.random.random((years.size, samples.size, len(patients))) #(10, 20, 3)

# Reshape data to 2 dimensions
maj_dim = 1
for dim in A_3D.shape[:-1]:
    maj_dim = maj_dim*dim
new_dims = (maj_dim, A_3D.shape[-1])
A_3D = A_3D.reshape(new_dims)

# Create the MultiIndex from years, samples and patients.
midx = pd.MultiIndex.from_product([years, samples])
# Note that Cartesian product order is the same as the 
# C-order used by default in ``reshape``.

# Create sample data for each patient, and add the MultiIndex.
patient_data = pd.DataFrame(data = A_3D, 
                            index = midx,
                            columns = patients)

patient_data.head()

patient_0  patient_1  patient_2
2000 0   0.238845   0.948436   0.954822
     1   0.343249   0.028403   0.402137
     2   0.497765   0.316694   0.589415
     3   0.792957   0.407125   0.660466
     4   0.124491   0.403153   0.097867

In [ ]:
#Set 3 axis labels/dims
years = BOOKIDS
samples = RESULTS
patients = SPALTEN

#Create random 3D array to simulate data from dims above
A_3D = np.random.random((years.size, samples.size, len(patients))) #(10, 20, 3)

# Reshape data to 2 dimensions
maj_dim = 1
for dim in A_3D.shape[:-1]:
    maj_dim = maj_dim*dim
new_dims = (maj_dim, A_3D.shape[-1])
A_3D = A_3D.reshape(new_dims)

# Create the MultiIndex from years, samples and patients.
midx = pd.MultiIndex.from_product([years, samples])
# Note that Cartesian product order is the same as the 
# C-order used by default in ``reshape``.

# Create sample data for each patient, and add the MultiIndex.
patient_data = pd.DataFrame(data = A_3D, 
                            index = midx,
                            columns = patients)

patient_data.head()